In [ ]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
symbol = 'BAC'
bac = yf.download(symbol, start='2022-01-01', interval='1d')
# bac

In [ ]:
import ta
data = bac.copy()
data['SMA1'] = data['Close'].rolling(window=42).mean()
data['SMA2'] = data['Close'].rolling(window=252).mean()

data['RSI'] = ta.momentum.RSIIndicator(data['Close']).rsi()
macd = ta.trend.MACD(data['Close'])
data['MACD'] = macd.macd()
data['MACD_signal'] = macd.macd_signal()

data.dropna(inplace=True)

data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

x = data.drop(columns='Close')
y = data['Close']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False, random_state=42)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
plt.figure(figsize=(14, 7))
plt.plot(y_test.index, y_test, label='Actual Close Price')
plt.plot(y_test.index, y_pred, label='Predicted Close Price')
plt.legend()
plt.show()